# Telco Customer Churn

link: https://www.kaggle.com/datasets/blastchar/telco-customer-churn

## Bibliotecas 

In [86]:
# manipulação de dados
import pandas as pd
import numpy as np

# pre-processamento
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

# separação treino e teste
from sklearn.model_selection import train_test_split

# modelos
from sklearn.linear_model import LogisticRegression

# metricas
from sklearn.metrics import accuracy_score, confusion_matrix

## Leitura dos dados 

In [3]:
df_raw = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Verificação dos dados

### tamanho dos dados

In [13]:
print('Total de linhas: ', df_raw.shape[0])
print('Total de colunas: ', df_raw.shape[1])

Total de linhas:  7043
Total de colunas:  21


### vertificar dados faltantes

In [6]:
df_raw.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

### verificar dados duplicados

In [8]:
df_raw.duplicated().sum()

0

### tipos dos dados

In [9]:
df_raw.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

## Seleção de features

**Não precisando do customerID**

In [51]:
df = df_raw.copy()
# não precisamos do customerID
df = df.drop('customerID', axis = 1)
#df = df.dropna(subset=['TotalCharges'])

# a variavel TotalCharges está como categorico
df['TotalCharges'] = df['TotalCharges'].replace(' ', 0).astype(float)


In [72]:
# target
#y = df['Churn']
y = pd.get_dummies(df['Churn'], drop_first = True)
y['Target'] = y['Yes']
y.drop('Yes', axis = 1, inplace=True)

# x categorico
onehotencoder = OneHotEncoder()
variaveis_categoricas = df.select_dtypes(include=['object']).drop('Churn', axis=1)
x_categorico = onehotencoder.fit_transform(variaveis_categoricas).toarray()
x_categorico = pd.DataFrame(x_categorico)

# x numerico
variaveis_numericas = df.select_dtypes(include=['int','float'])
x_numerico = variaveis_numericas

# variavel categorica
x_all = pd.concat([x_categorico, x_numerico], axis=1)
scaler = MinMaxScaler()
x = scaler.fit_transform(x_all)
x


C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


array([[1.        , 0.        , 0.        , ..., 0.01388889, 0.11542289,
        0.00343704],
       [0.        , 1.        , 1.        , ..., 0.47222222, 0.38507463,
        0.21756402],
       [0.        , 1.        , 1.        , ..., 0.02777778, 0.35422886,
        0.01245279],
       ...,
       [1.        , 0.        , 0.        , ..., 0.15277778, 0.11293532,
        0.03989153],
       [0.        , 1.        , 0.        , ..., 0.05555556, 0.55870647,
        0.03530306],
       [0.        , 1.        , 1.        , ..., 0.91666667, 0.86965174,
        0.78810105]])

In [74]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25)

In [75]:
x_train.shape

(5282, 45)

In [76]:
x_test.shape

(1761, 45)

In [77]:
y_train.shape

(5282, 1)

In [78]:
y_test.shape

(1761, 1)

## Regressão logistica

In [81]:
logistic_model = LogisticRegression()
logistic_model.fit(x_train, y_train)

C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [83]:
y_pred = logistic_model.predict(x_test)
y_pred

array([0, 1, 0, ..., 0, 0, 0], dtype=uint8)

In [85]:
accuracy_score(y_test, y_pred)

0.817717206132879

In [88]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1152,  137],
       [ 184,  288]], dtype=int64)